In [ ]:
from pylab import *

import skimage
from skimage import util

from scipy import ndimage
from skimage import restoration

import numpy as np

In [ ]:
img = skimage.img_as_float(imread('sekvence/lena.tif'))

img_gaus_noise_0p001 = util.random_noise(img, mode='gaussian', mean=0, var=0.001)
img_gaus_noise_0p01  = util.random_noise(img, mode='gaussian', mean=0, var=0.01)
img_impulse_noise    = util.random_noise(img, mode='s&p', amount=0.2)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,16), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_gaus_noise_0p001, cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Gausov šum - var=0.001')
ax[2].imshow(img_gaus_noise_0p01, cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Gausov šum - var=0.01')
ax[3].imshow(img_impulse_noise, cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Impulsni šum');

In [ ]:
img = skimage.img_as_float(imread('sekvence/lena.tif'))

noise_var = 0.01
img_noise = util.random_noise(img, 'gaussian', var=noise_var)

filt_avg = ones([7,7])/(7**2)

img_local_avg = ndimage.correlate(img_noise, filt_avg)
img_sqr_local_avg = ndimage.correlate(img_noise**2, filt_avg)

img_local_var = img_sqr_local_avg - img_local_avg**2

weight = noise_var/img_local_var
weight[weight>1]=1

img_est = img_noise + weight*(img_local_avg-img_noise)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,16), dpi=120)
ax = axes.ravel()

ax[0].imshow(img_noise, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(img_local_avg, cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Usrednjena slika - filtar 7x7')
ax[2].imshow(weight, cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Mapa težina')
ax[3].imshow(img_est, cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Izlaz nakon adaptivnog usrednjavanja');

In [ ]:
img = skimage.img_as_float(imread('sekvence/road_blur.tif'))

plt.figure(figsize=(8,8), dpi=120); imshow(img, cmap='gray');

In [ ]:
import scipy.io

DATA = scipy.io.loadmat('sekvence/h_blur.mat')
H_blur = DATA['H_blur']

plt.figure(figsize=(8,8), dpi=120); imshow(H_blur, cmap='gray');

In [ ]:
img_fft = fftshift(fft2(img))

img_fft_est = img_fft/H_blur

plt.figure(figsize=(8,8), dpi=120); imshow(log(1+abs(img_fft_est)), cmap='gray');

In [ ]:
def lpfilter(filt_type, Ny, Nx, sigma, n=1):
    
    if (Ny%2 == 0):
        y = np.arange(0,Ny) - Ny/2 + 0.5
    else:
        y = np.arange(0,Ny) - (Ny-1)/2
    
    if (Nx%2 == 0):
        x = np.arange(0,Nx) - Nx/2 + 0.5
    else:
        x = np.arange(0,Nx) - (Nx-1)/2

    
    X, Y = meshgrid(x, y)
    
    D = np.sqrt(np.square(X) + np.square(Y))
    
    if filt_type == 'gaussian':
        filter_mask = exp(-np.square(D)/(2*np.square(sigma)))
    elif filt_type == 'btw':
        filter_mask = 1/(1+(D/sigma)**(2*n))
    elif filt_type == 'ideal':
        filter_mask = ones([Ny, Nx])
        filter_mask[D>sigma] = 0
    else:
        print('Greška! Nije podržan tip filtra: ', filt_type)
        return
    
    return filter_mask

In [ ]:
[Ny, Nx] = shape(img)
lp_filt_freq = lpfilter('ideal', Ny, Nx, 130)

img_fft_est_filt = (img_fft/H_blur)*lp_filt_freq

img_est = real(ifft2(ifftshift(img_fft_est_filt)))
img_est = np.clip(img_est, 0, 1)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,16), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(log(1+abs(img_fft_est)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Spektar nakon inverznog filtriranja')
ax[2].imshow(log(1+abs(img_fft_est_filt)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Spektar nakon ograničenja opsega')
ax[3].imshow(img_est, cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Estimirani izlaz');

In [ ]:
img = skimage.img_as_float(imread('sekvence/road_blur.tif'))
[Ny, Nx] = shape(img)

img_fft = fftshift(fft2(img))

k = 1e-4
W = (abs(H_blur)**2)/(abs(H_blur)**2 + k)

img_fft_est_filt = (img_fft/H_blur)*W

img_est = real(ifft2(ifftshift(img_fft_est_filt)))
img_est = np.clip(img_est, 0, 1)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,16), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(log(1+abs(img_fft_est)), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Spektar nakon inverznog filtriranja')
ax[2].imshow(log(1+abs(img_fft_est_filt)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Spektar nakon Vinerovog filtra')
ax[3].imshow(img_est, cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Estimirani izlaz');

In [ ]:
img = skimage.img_as_float(imread('sekvence/road_motion.tif'))

DATA = scipy.io.loadmat('sekvence/h_motion.mat')
H_motion = DATA['H_motion']


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,8), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_title('Ulazna slika')
ax[1].imshow(abs(H_motion), cmap='gray'); ax[1].set_title('Spektar degradacione funkcije');

In [ ]:
img_fft = fftshift(fft2(img))

k = 1e-4
W = (abs(H_motion)**2)/(abs(H_motion)**2 + k)

img_fft_est = (img_fft/H_motion)*W

img_est = real(ifft2(ifftshift(img_fft_est)))
img_est = np.clip(img_est, 0, 1)

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,16), dpi=120)
ax = axes.ravel()

ax[0].imshow(img, cmap='gray'); ax[0].set_axis_off(); ax[0].set_title('Ulazna slika')
ax[1].imshow(abs(H_motion), cmap='gray'); ax[1].set_axis_off(); ax[1].set_title('Spektar degradacione funkcije')
ax[2].imshow(log(1+abs(img_fft_est)), cmap='gray'); ax[2].set_axis_off(); ax[2].set_title('Spektar nakon Vinerovog filtra')
ax[3].imshow(img_est, cmap='gray'); ax[3].set_axis_off(); ax[3].set_title('Estimirani izlaz');